In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from datetime import datetime
from numpy import pi, sin, cos, tan, arcsin, arccos, arctan, sqrt
from sko.PSO import PSO

plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

In [2]:
PHI = 39.4 / 180 * pi
ST = [9.0, 10.5, 12.0,  13.5, 15.0]
D = [(datetime(2023,i,21)-datetime(2023,3,21)).days for i in range(1, 13)]
h_t = 80 # 集热器中心高度
h_max = 84 # 塔顶端高度
h_m = 4  # 定日镜中心高度
m_length, m_width = 6, 6
m_x_bound, m_y_bound = [-3,3], [-3,3] # 定日镜坐标边界
t_x_bound, t_z_bound = [-3.5, 3.5], [0, h_max]

## 常用函数

In [3]:
def cal_distance(m1xy, m2xy):
    # 计算二维距离
    dis = np.linalg.norm(m1xy - m2xy)
    return dis

In [4]:
def in_bound(x, bound):
    # 判断点是否出了边界
    if (x < bound[0] or x > bound[1]):
        return False
    return True

## 计算太阳高度角，方位角

In [5]:
def cal_omega(st):
    # 计算太阳时角
    return pi / 12 * (st - 12)

def cal_delta(d):
    # 计算太阳赤纬角
    sin_delta = sin(2*pi*d/365) * sin(2*pi*23.45/360)
    return arcsin(sin_delta)

def scale_in_min_max(num, min=-1, max=1):
    # 将结果限制在范围内
    if (num < min): return min
    elif (num > max): return max
    else: return num

def cal_alpha_and_gamma_s(d, st):
    # 计算太阳高度角和方位角
    omega = cal_omega(st)
    delta = cal_delta(d)
    sin_alpha_s = cos(delta)*cos(PHI)*cos(omega) + sin(delta)*sin(PHI)
    sin_alpha_s = scale_in_min_max(sin_alpha_s)
    alpha_s = arcsin(sin_alpha_s)
    cos_gamma_s = (sin(delta) - sin(alpha_s) * sin(PHI)) / ((cos(alpha_s) * cos(PHI)))
    cos_gamma_s = scale_in_min_max(cos_gamma_s)
    gamma_s = arccos(cos_gamma_s)
    if (st > 12): 
        gamma_s = 2*pi - gamma_s
    return alpha_s, gamma_s

In [6]:
alpha_mat = np.zeros((12, 5))
gamma_mat = np.zeros((12, 5))
for i, d in enumerate(D):
    for j, st in enumerate(ST):
        alpha, gamma = cal_alpha_and_gamma_s(d, st)
        alpha_mat[i,j] = alpha
        gamma_mat[i,j] = gamma

## 计算定日镜俯仰角，方位角

In [7]:
def cal_S_r(mirror_xyz):
    # 计算反射光线单位向量
    O = np.array([0 ,0, h_t])
    O_A = np.array(mirror_xyz)
    v = O - O_A
    v = v / np.linalg.norm(v)
    return v

def cal_S_i(alpha, gamma):
    # 计算入射光线单位向量
    # alpha, gamma是弧度制
    x = -cos(alpha) * cos(gamma-pi/2)
    y = cos(alpha) * sin(gamma-pi/2)
    z = -sin(alpha)
    return np.array([x, y, z])

In [8]:
# 计算定日镜法向量
S_i_mat = np.zeros((12, 5, 3))
for i, d in enumerate(D):
    for j, st in enumerate(ST):
        alpha, gamma = cal_alpha_and_gamma_s(d, st)
        S_i = cal_S_i(alpha, gamma)
        S_i_mat[i,j] = S_i

## 计算简单效率

In [9]:
# 计算大气透射率
def cal_ita_at(mirror_xyz):
    d = np.linalg.norm(mirror_xyz - np.array([0, 0, h_t]))
    return 0.99321 - 0.0001176*d + 1.97*10**(-8)*d**2

## 计算DNI

In [10]:
G0 = 1.366
H = 3
a = 0.4237 - 0.00821*(6-H)**2
b = 0.5055 + 0.00595*(6.5-H)**2
c = 0.2711 + 0.01858*(2.5-H)**2
DNI_mat = np.zeros((12, 5))
for i, _ in enumerate(D):
    for j, _ in enumerate(ST):
        alpha, gamma = cal_alpha_and_gamma_s(D[i], ST[j])
        DNI_mat[i,j] = G0*(a + b*pow(np.e,-c/sin(alpha)))

In [11]:
def cal_mod_length(x):
    return sqrt(x[0]**2+x[1]**2)

In [ ]:
slice_num = 350
max_bound = 700
x = np.linspace(-max_bound, max_bound, slice_num)
y = np.linspace(-max_bound, max_bound, slice_num)
z = np.ones((slice_num)) * 5
xyz = []
for i in range(slice_num ):
    for j in range(slice_num):
        if cal_mod_length((x[i],y[j])) > 100:
            xyz.append([x[i], y[j], z[i]])
mirrors_xyz = np.array(xyz)
mirrors_xy = mirrors_xyz[:,:2]
m_num = mirrors_xyz.shape[0]
S_r_mat = np.zeros((12, 5, *mirrors_xyz.shape))
S_n_mat = np.zeros((12, 5, *mirrors_xyz.shape))
for i, d in enumerate(D):
    for j, st in enumerate(ST):
        S_i = S_i_mat[i,j]
        for k, mirror_xyz in enumerate(mirrors_xyz):
            S_r = cal_S_r(mirror_xyz)
            S_n = (S_r - S_i) / np.linalg.norm(S_r - S_i)
            S_r_mat[i,j,k] = S_r
            S_n_mat[i,j,k] = S_n
# 计算余弦效率
ita_cos_mat = np.zeros((12, 5, m_num))
for i, d in enumerate(D):
    for j, st in enumerate(ST):
        alpha, gamma = alpha_mat[i,j], gamma_mat[i,j]
        S_i = S_i_mat[i,j]
        neg_S_i = -S_i
        for k, mirror_xyz in enumerate(mirrors_xyz):
            ita_cos = S_n_mat[i,j,k] @ neg_S_i
            ita_cos_mat[i,j,k] = ita_cos 
# 计算大气透射率
ita_at_mat = np.zeros(m_num)
for k, mirror_xyz in enumerate(mirrors_xyz):
    ita_at_mat[k] = cal_ita_at(mirror_xyz)
ita_mat = np.zeros((12, 5, m_num))
E_mat = np.zeros((m_num))
for i, _ in enumerate(D):
    for j, _ in enumerate(ST):
        for k, _ in enumerate(mirrors_xyz):
            ita_mat[i,j,k] = ita_at_mat[k] * ita_cos_mat[i,j,k]
            E_mat[k] += DNI_mat[i,j] * ita_mat[i,j,k]  
E_mat /= 60      
for k, mirror_xyz in enumerate(mirrors_xyz):
    if cal_mod_length(mirror_xyz[:2]) < 100:
        E_mat[k] = 0

In [ ]:
title = "镜场年平均能量传输效率"
cm = matplotlib.colormaps['rainbow']
sc = plt.scatter(mirrors_xy[:,0], mirrors_xy[:,1], c=ita_mat.mean(axis=0).mean(axis=0), s=1, cmap=cm)
plt.title(title)
plt.colorbar(sc)
plt.savefig(title + ".png", dpi=200)
plt.show()

In [ ]:
ita_mat = ita_mat.mean(axis=0).mean(axis=0)

In [ ]:
R_field = 350
mont_bound = 250
mont_num = 50
x_mont = np.linspace(-mont_bound, mont_bound, mont_num)
y_mont = np.linspace(-mont_bound, mont_bound, mont_num)
z_mont = np.ones((mont_num)) * 5
xyz_mont = np.zeros((mont_num**2 , 3))
for i in range(mont_num):
    for j in range(mont_num):
        xyz_mont[i*mont_num +j] = [x_mont[i], y_mont[j], z_mont[i]]

sm_mat = np.zeros((xyz_mont.shape[0]))
for i, (x, y, z) in enumerate(xyz_mont):
    sm = 0
    if cal_mod_length((x,y)) <= 250:
        for k, mirror_xyz in enumerate(mirrors_xyz):
            if cal_distance(mirror_xyz[:2], [x,y]) < R_field:
                sm += ita_mat[k]
    sm_mat[i] = sm

In [ ]:
def find_max(xy_mont):
    R_field = 350
    x, y = xy_mont
    sm = 0
    for k, mirror_xyz in enumerate(mirrors_xyz):
        if cal_distance(mirror_xyz[:2], [x,y]) < R_field:
            sm += ita_mat[k]
    return -sm

In [ ]:
constraint_ueq = (
    lambda x: cal_mod_length((x[0],x[1])) - 250,
)
pso = PSO(func=find_max, n_dim=2, pop=40, max_iter=20, lb=[-mont_bound, -mont_bound], ub=[mont_bound, mont_bound], constraint_ueq=constraint_ueq)
pso.run()
print('best_x is ', pso.gbest_x, 'best_y is', pso.gbest_y)

In [ ]:
plt.plot(pso.gbest_y_hist)
plt.show()